In [1]:
import sys
sys.path.append('../') # add this project path to the system path to import own module

from Util.util import Constant, getMilliTime, Buyer
from Util import util
import time
import requests
import pickle
from Ip.ipCheck import IpPool, Ip
from Account.accountCheck import AccountPool, Account
import threading
from hyper.contrib import HTTP20Adapter
import random
import math

In [2]:
class MyThread(threading.Thread):
    def __init__(self, threadID, account, sleepSec, goods_id):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.account = account
        self.sleepSecRange = [sleepSec / 2, sleepSec] # sleep from accountNum / 2 to accountNum sec.
        self.goods_id = goods_id
        self.buff_req_headers = { # request header for accessing market in buff
            ':authority': 'buff.163.com',
            ':scheme': 'https',
            'accept': 'application/json, text/javascript, */*; q=0.01',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Linux"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mod': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
            'x-requested-with': 'XMLHttpRequest'
        } 
        self.sessions = requests.session() # the session for accessing buff
        self.sessions.mount(Constant.domain, HTTP20Adapter())
        self.sessions.headers = self.buff_req_headers

    def get_goods_data(self):
        # 裂空武器箱:781534
        _path = Constant.getGoodsAddr(self.goods_id, getMilliTime())
        _url = Constant.domain + _path
        _header = {
            ':method': 'GET',
            ':path': _path,
            'cookie': self.account.cookie + 'csrf_token=' + self.account.csrf_token,
            'referer': Constant.domain + '/goods/' + self.goods_id,
        }
        resp = self.sessions.get(_url, headers = _header, proxies = self.account.ipObj.proxies)
        # print(resp.headers)
        try:
            self.account.csrf_token = resp.headers.get(b'set-cookie').decode("utf-8").split(';')[0].split('=')[1]
        except:
            pass
        return resp
        
    def run(self):
        print ("Starting thread: " + self.threadID + ' ip : ' + self.account.ipObj.proxies['http'] + ' phone number : ' + self.account.buff_phone + '\n')
        
        while(True):
            try:
                resp = self.get_goods_data()
                item = resp.json()['data']['items'][0]
                print(item['price'])
                if(float(item['price']) < 9.3):
                    Buyer.buy_goods(self.goods_id, item['id'], item['price'])
                    print('buy the goods, id: ' + self.goods_id + 'price is :' + item['price'] + '\n')
                    Buyer.sendMessage(item['price'])
                    # print('get_goods_data: \n')
                    # print(item)
                
                time.sleep(self.sleepSecRange[1])
                # time.sleep(10)
            except Exception as e:
                print('exception in the MyThread class: \n')
                print(e)
                continue

class Main(object):
    def __init__(self):
        self.account_list = self.getAccountPool()
        self.account_len = len(self.account_list)
        self.ipPool = self.getIpPool()
        self.run()
        
    def run(self):
        print('accounts number: ' + str(self.account_len))
        for i in range(0, self.account_len):
            if not self.account_list[i].is_login:
                continue
            if not self.account_list[i].is_market:
                continue
            time.sleep(1)
            is_alive, lag_time = util.checkAlive(self.account_list[i].ipObj.proxies)
            if not is_alive:
                print('ip ' + self.account_list[i].ipObj.ip + ' is not alive \n')
                self.account_list[i].ipObj = self.ipPool.getAnIp()
            if lag_time > 20:
                print('ip ' + self.account_list[i].ipObj.ip + ' lag too long \n')
                self.account_list[i].ipObj = self.ipPool.getAnIp()
            
            MyThread('Thread ' + str(i), self.account_list[i], self.account_len, '45237').start()
    
    def getIpPool(self):
        return IpPool(0)

    def getAccountPool(self):
        with open(Constant.account_pool_dir, 'rb') as fin: 
            accountPool = pickle.load(fin)
            return self.upateAccountPool(accountPool)
    
    def upateAccountPool(self, accountPool):
        _now = time.time()
        for account in accountPool:
            if (math.ceil((_now - account.login_time) / 86400) > 9):# account will be expired soon, relogin here
                account.relogin()
        return accountPool

In [3]:
Main()

accounts number: 19
Starting thread: Thread 0 ip : http://140.250.92.75:18558 phone number : 17056635880

9.59
Starting thread: Thread 1 ip : http://49.88.215.247:16130 phone number : 17022402286

9.59
Starting thread: Thread 2 ip : http://175.170.37.186:21261 phone number : 16737138654

9.5
Starting thread: Thread 3 ip : http://175.173.220.42:15734 phone number : 16571003870

9.5
Starting thread: Thread 4 ip : http://119.112.203.157:22910 phone number : 17050573086

9.5
Starting thread: Thread 5 ip : http://42.177.116.79:21006 phone number : 16215707729

9.49
Starting thread: Thread 6 ip : http://175.154.212.114:21559 phone number : 16537761354

9.49
Starting thread: Thread 7 ip : http://14.20.190.197:16503 phone number : 17037448898

9.49
Starting thread: Thread 8 ip : http://111.224.220.29:17736 phone number : 17530549615

9.49
Starting thread: Thread 9 ip : http://117.92.131.42:20383 phone number : 16537928412

9.49
Starting thread: Thread 11 ip : http://14.20.190.57:19590 phone nu

IndexError: list index out of range

9.49
9.49
9.49
9.49
9.49
9.49
9.49
7.39
buy the goods, id: 900464price is :7.39

9.55
9.55
9.55
9.55
9.49
9.49
9.49
9.49
9.49
9.49
9.49
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.5
9.5
9.5
9.5
9.5
9.5
9.5
9.5
9.5
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.58
9.58
9.58
9.58
9.58
9.58
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
9.59
